In [3]:
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers

import numpy as np

In [4]:
bottleneck = np.load('bottleneck_features/DogResnet50Data.npz')
train_Resnet50 = bottleneck['train']
valid_Resnet50 = bottleneck['valid']
test_Resnet50 = bottleneck['test']

model = Sequential()

model.add(GlobalAveragePooling2D(input_shape = train_Resnet50.shape[1:]))
model.add(Dropout(0.5))
model.add(Dense(133,activation = 'softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 133)               272517    
Total params: 272,517
Trainable params: 272,517
Non-trainable params: 0
_________________________________________________________________


In [5]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from glob import glob

def load_dataset(path):
    data = load_files(path)
    dog_file = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']),133)
    return dog_file,dog_targets

# load train,validation and test datasets
train,train_target = load_dataset('dogImages/train')
test,test_target = load_dataset('dogImages/test')
valid,valid_target = load_dataset('dogImages/valid')

In [6]:
dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train, valid, test])))
print('There are %d training dog images.' % len(train))
print('There are %d validation dog images.' % len(valid))
print('There are %d test dog images.'% len(test))

There are 133 total dog categories.
There are 8351 total dog images.

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [8]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

checkp = ModelCheckpoint(filepath = 'weights.best.dog.hdf5',
                        verbose = 1, save_best_only = True)

model.fit(train_Resnet50,train_target,
         validation_data = (valid_Resnet50,valid_target),
         epochs = 12, batch_size = 20 , callbacks = [checkp],verbose = 1)

Train on 6680 samples, validate on 835 samples
Epoch 1/12
6680/6680 [==============================] - 4s 587us/step - loss: 0.2070 - accuracy: 0.9320 - val_loss: 0.7684 - val_accuracy: 0.7904

Epoch 00001: val_loss improved from inf to 0.76841, saving model to weights.best.dog.hdf5
Epoch 2/12
6680/6680 [==============================] - 4s 547us/step - loss: 0.1669 - accuracy: 0.9484 - val_loss: 0.7366 - val_accuracy: 0.7856

Epoch 00002: val_loss improved from 0.76841 to 0.73656, saving model to weights.best.dog.hdf5
Epoch 3/12
6680/6680 [==============================] - 4s 571us/step - loss: 0.1554 - accuracy: 0.9472 - val_loss: 0.7633 - val_accuracy: 0.7892

Epoch 00003: val_loss did not improve from 0.73656
Epoch 4/12
6680/6680 [==============================] - 4s 545us/step - loss: 0.1435 - accuracy: 0.9504 - val_loss: 0.7695 - val_accuracy: 0.8060

Epoch 00004: val_loss did not improve from 0.73656
Epoch 5/12
6680/6680 [==============================] - 4s 566us/step - loss: 0

In [9]:
score = model.evaluate(test_Resnet50,test_target,verbose = 0)
print(f'loss of test {score[0]}, accurace_test {score[1]}')

loss of test 0.9199533314225776, accurace_test 0.8133971095085144
